# Resumen
Este cuaderno digital interactivo tiene como objetivo demostrar las relaciones entre las propiedades fisico-químicas de la vegetación y el espectro solar.

Para ello haremos uso de modelos de simulación, en particular de modelos de transferencia radiativa tanto a nivel de hoja individual como a nivel de dosel vegetal.

# Instrucciones
Lee con detenimiento todo el texto, y sigue sus instrucciones.

Una vez leida cada sección de texto ejecuta la celda de código siguiente (marcada como `In []`) presionando el icono de `Run`/`Ejecutar` o presionando en el teclado ALT + ENTER. Aparecerá una interfaz gráfica con la que poder realizar las tareas asignadas.

Como ejemplo ejectuta la siguiente celda para importar todas  las librerías necesarias para el correcto funcionamiento del cuaderno. Una vez ejecutada debería aparecer un mensaje de agradecimiento.

In [ ]:
%matplotlib inline
from ipywidgets import FloatSlider, FloatRangeSlider, Dropdown, interact, interactive, fixed
from IPython.display import display
from functions.prosail_and_spectra import *
slide_kwargs = {"continuous_update": False}

# Espectro de una hoja
Las propiedades espectrales de una hoja (tanto su transmisividad, su reflectividad y su absortividad) dependen de su concentración de pigmentos, de su contenido de agua, su peso específico y la estructura interna de sus tejidos. 

Vamos a usar el modelo ProspectD, el cual es una simplificación de la realidad en la que simula el espectro mediante la concentración de clorofilas (`Cab`), carotenoides (`Car`), antocianinos (`Ant`), así como el peso de agua y por unidad de supeficie (`Cw`) y el peso del resto de la materia seca (`Cm`) que engloba las celulosas, ligninas (responsables principales de la biomasa foliar) y otros componentes proteicos. También incluye un parámetro semi-empírico que representa otros pigmentos responsables del color de las hojas senescentes y enfermas. Además con el fin de simular hojas con distintas estructuras celulares incluye un último parámetro (`N_leaf`) que emula las distitas capas y tejidos celulares de la hoja.

Ejecuta la siguiente célula y verás un espectro típico de la hoja. El gráfico muestra tanto la reflectividad (en el eje y) como la transmisividad (en el eje secundario y, con valores invertidos) y la absortividad (como el espacio entre las dos curvas de reflectividad y transmisividad) $\rho + \tau + \alpha = 1$.

Presta atención a cómo y en qué regiones cambia el espectro según el parámetro que modifiques.
* Haz variar la clorofila. 
* Haz variar el contenido de agua
* Haz variar la materia seca
* Haz variar los pigmentos marrones desde un valor de 0 (hoja sana) a valores mayores (hoja enferma o seca)

In [ ]:
w_nleaf = FloatSlider(value=1.5, min=1, max=4, step=0.01, description='N:', **slide_kwargs)
w_cab = FloatSlider(value=40, min=0, max=110, step=1, description='Cab ($\mu$g/cm²):', **slide_kwargs)
w_car = FloatSlider(value=10, min=0, max=40, step=1, description='Car ($\mu$g/cm²):', **slide_kwargs)
w_ant = FloatSlider(value=1, min=0, max=40, step=1, description='Ant ($\mu$g/cm²):', **slide_kwargs)
w_cbrown = FloatSlider(value=0, min=0, max=3, step=0.05, description='Cbrown (-):', **slide_kwargs)
w_cw = FloatSlider(value=0.0053, min=0.000, max=0.050, step=0.001, description='Cw (g/cm²):', 
                   readout_format='.3f', **slide_kwargs)
w_cm = FloatSlider(value=0.0114, min=0.001, max=0.040, step=0.001, description='Cm (g/cm²):', 
                   readout_format='.3f', **slide_kwargs)
w_rho_leaf = interactive(update_prospect_spectrum, N_leaf=w_nleaf, Cab=w_cab, Car=w_car, Ant=w_ant, 
                     Cbrown=w_cbrown, Cw=w_cw, Cm=w_cm)
display(w_rho_leaf)

# Espectro del suelo
El espectro del dosel o de la supeficie vegetal no sólo depende del espectro y las propiedades de las hojas, sino que también de la propia estructura del dosel así como del suelo. En particular en doseles abiertos o poco densos, como el las primeras fases fenológicas el comportamiento espectral del suelo puede influir de manera muy importante en la señal espectral que capten los sensores de teledetección.

El espectro del suelo depende de varios factores, como son su composición mineralógica, su textura y densidad así como su humedad superficial. 

Ejectuta la siguiente celda y mira los distintas características espectrales de distintos tipos de suelo. En general observa lo diferente que puede ser un espectro de suelo en comparación con el de una hoja. Esto es clave a la hora de clasificar tipos de coberturas mediante teledetección así como cuantificar el vigor/densidad vegetal del cultivo.

In [ ]:
w_soil = Dropdown(options=SOIL_TYPES, value=SOIL_TYPES[0], description='Soil Type')
w_rho_soil = interactive(update_soil_spectrum, soil_name=w_soil)
display(w_rho_soil)

# Espectro del dosel
Finalmente,integrando la firma espectral de una hoja y del suelo subyacente, podemos obtener el espectro de un dosel vegetal. 

El espectro de la superficie vegetal además depende de la estructura del dosel, principalmente de la cantidad de hojas por unidad de superficie (definido aquí como el ïndice de Área Foliar) y de cómo estas hojas se orientan con respecto a la vertical. Además, dado que se produce una interacción de la luz incidente y reflejada entre el volumen de hojas y el suelo, la posición del sol y del sensor influyen en la señal espectral que obtengamos.

Para esta parte cobinaremos el modelo de transferencia ProspectD para simular el espectro de una hoja con otro modelo de trasnferencia a nivel de dosel (4SAIL). Este último modelo considera la superficie vegetal como una capa horizontal y verticalmente homogéna, por lo que se recomienda cautela en su aplicación en doseles arbóreos heterogéneos.

Ejecuta la siguente celda y mira cómo los espectros de hoja y suelo se integran para obtener un espectro de la superficie vegetal.

In [ ]:
w_lai = FloatSlider(value=1., min=0, max=10, step=0.1, description='LAI (m²/m²):', **slide_kwargs)
w_hotspot = FloatSlider(value=0.01, min=0, max=1, step=0.01, description='hotspot (-):', **slide_kwargs)
w_leaf_angle = FloatSlider(value=57., min=0, max=90, step=1, description='Leaf Angle (deg.):', **slide_kwargs)
w_sza = FloatSlider(value=35., min=0, max=89, step=1, description='SZA (deg.):', **slide_kwargs)
w_vza = FloatSlider(value=0, min=0, max=89, step=1, description='VZA (deg.):', **slide_kwargs)
w_psi = FloatSlider(value=0, min=0, max=180, step=1, description='PSI (deg.):', **slide_kwargs)
w_skyl = FloatSlider(value=0.1, min=0, max=1, step=0.01, description='skyl (-):', **slide_kwargs)
w_rho_canopy = interactive(update_4sail_spectrum,{'manual': True, "manual_name": "Generar Espectro"},
                           lai=w_lai, hotspot=w_hotspot, leaf_angle=w_leaf_angle, sza=w_sza, vza=w_vza,
                           psi=w_psi, skyl=w_skyl, leaf_spectrum=fixed(w_rho_leaf), soil_spectrum=fixed(w_rho_soil))
display(w_rho_canopy)

# Sensibilidad de los parámetros
En esta tarea podrás ver el comportamiento espectral de la vegetación según varían los parámetros fisico-químicos de la vegetación así como su sensibilidad a las condiciones de observación e iluminación.

Para ello vamos a realizar un análisis de sensibilidad variando un sólo parámetro a la vez, mientras que el resto de los parámetros permanecerán constantes. Selecciona qué parámetro quieres analizar y el rango de valores máximo y mínimo que quieras que tenga. A continuación puedes variar los valores individuales para el resto de los parámetros. Finalmente presiona `Get Spectra` para generar un gráfico mostrando cómo varía el espectro con respecto al parámetro analizado. 

In [ ]:
w_param = Dropdown(options=RANGE_DICT.keys(), value="LAI", description='Variable a evaluar')
w_range = FloatRangeSlider(value=RANGE_DICT["LAI"], min=RANGE_DICT["LAI"][0], max=RANGE_DICT["LAI"][1],
                          description="Rango", readout_format='.1f')

def _on_param_change(args):
    var = args["new"]
    if var == "Cm" or var == "Cw":
        w_range.readout_format = '.3f'
        w_range.step = 0.001
    else:
        w_range.readout_format='.1f'
        w_range.step = 0.1
        
    if RANGE_DICT[var][1] < w_range.min:
        w_range.min = RANGE_DICT[var][0]
        w_range.max = RANGE_DICT[var][1]
    else:
        w_range.max = RANGE_DICT[var][1]
        w_range.min = RANGE_DICT[var][0]
    w_range.value = RANGE_DICT[var]

w_param.observe(_on_param_change, 'value', type="change")
w_sensitivity = interactive(prosail_sensitivity, {'manual': True, "manual_name": "Generar Espectros"},
                            N_leaf=w_nleaf, Cab=w_cab, Car=w_car, Ant=w_ant,Cbrown=w_cbrown, Cw=w_cw, Cm=w_cm,
                            lai=w_lai, hotspot=w_hotspot, leaf_angle=w_leaf_angle, sza=w_sza, vza=w_vza,
                            psi=w_psi, skyl=w_skyl, soil=w_soil, var=w_param, value_range=w_range)
display(w_sensitivity)